In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

<p>1. Read the train dataset</p>

In [2]:
df = pd.read_csv('../Data/train_ver2_product_added.csv')

/home/pablo/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<p>2. Set the age feature due to the results of the exploratoty analysis.</p>

In [3]:
df['age'] = df['age'].apply(lambda x: 1 if x <= 17 else 0)

<p>3. Select the useful features</p>

In [4]:
products_df = df.ix[:, df.iloc[:1,].filter(regex="ind_+.*ult.*").columns.values]
products_df['fecha_dato'] = df['fecha_dato']
products_df['product'] = df['product']
products_df['age'] = df['age']

<p>4. Free memory </p>

In [5]:
del df

<p>5. Split the train dataset in two datasets, one for train and the other for test. </p>

In [6]:
train_df = products_df[products_df['fecha_dato'] != '2016-05-28']
test_df = products_df[products_df['fecha_dato'] == '2016-05-28']

X = train_df.ix[:, train_df.columns != 'product']
y = train_df['product']
X_test = test_df.ix[:, test_df.columns != 'product']
y_test = test_df['product']

X = X.drop('fecha_dato', axis=1)
X_test = X_test.drop('fecha_dato', axis=1)

<p>6. Use a Random Forest Classifier for predict the products that a person will buy in the next month</p>

In [7]:
clfrf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
clfrf.fit(X, y)
Z_rf = clfrf.predict(X_test)

corrects = 0
y_test_array = y_test.as_matrix()
for i in range(len(Z_rf)):
    predicted_products = Z_rf[i].split()
    real_products = y_test_array[i].split()
    correct = np.intersect1d(predicted_products, real_products)
    if len(correct) > 0:
        corrects += 1
    
accuracy_rf = corrects / float(len(y_test_array))
print 'optimist accuracy: ' + str(accuracy_rf)

accuracy_rf = (Z_rf == y_test).sum() / float(len(y_test))
print 'pesimist accuracy: ' + str(accuracy_rf)

optimist accuracy: 0.816735922052
pesimist accuracy: 0.747599942685
